In [2]:
from bs4 import BeautifulSoup
import requests

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd
import re

# Get the unique movie list

In [3]:
df = pd.read_csv("all_movie_info.csv")
unique_movie = df.drop_duplicates(subset=['movie_name', 'movie_director','movie_year'])
unique_movie

,movie_name,movie_director,movie_year,movie_rank,critic_name,critic_org,critic_cn
0,Mulholland Drive,David Lynch,2001,1,Simon Abrams,Freelance film critic,US
1,In the Mood for Love,Wong Kar-wai,2000,2,Simon Abrams,Freelance film critic,US
2,The Tree of Life,Terrence Malick,2011,3,Simon Abrams,Freelance film critic,US
3,Yi Yi: A One and a Two,Edward Yang,2000,4,Simon Abrams,Freelance film critic,US
4,Goodbye to Language,Jean-Luc Godard,2014,5,Simon Abrams,Freelance film critic,US
...,...,...,...,...,...,...,...
1739,I Don't Want to Sleep Alone,Tsai Ming-liang,2006,10,Jake Wilson,Fairfax Media,Australia
1741,Kings and Queen,Arnaud Desplechin,2004,2,Stephanie Zacharek,Time Magazine,US
1761,The Hours,Stephen Daldry,2002,2,Raymond Zhou,China Daily,China
1762,The Sun Also Rises,Jiang Wen,2007,3,Raymond Zhou,China Daily,China


In [4]:
moviename_list = unique_movie.movie_name.tolist()
moviename_list

['Mulholland Drive',
 'In the Mood for Love',
 'The Tree of Life',
 'Yi Yi: A One and a Two',
 'Goodbye to Language',
 'The White Meadows',
 'Night Across the Street',
 'Certified Copy',
 'Sparrow',
 'Fados',
 'Eternal Sunshine of the Spotless Mind',
 'Syndromes and a Century',
 'Spirited Away',
 'The Act of Killing',
 'The Grand Budapest Hotel',
 'The New World',
 'The World',
 'Elephant',
 'Zero Dark Thirty',
 'A History of Violence',
 'Stories We Tell',
 'Casino Royale',
 'Tabu',
 'Snow White',
 'Frozen River',
 'Gosford Park',
 'Inception',
 "Pan's Labyrinth",
 'Caché',
 'Grizzly Man',
 '4 Months, 3 Weeks & 2 Days',
 'Holy Motors',
 'The Last of the Unjust',
 'There Will Be Blood',
 'The Piano Teacher',
 'Margaret',
 'American Psycho',
 'Lourdes',
 'Red Road',
 'Boyhood',
 'Tony Manero',
 'No Country For Old Men',
 'A Separation',
 'Finding Nemo',
 'Hero',
 'The Wolf of Wall Street',
 'Mother',
 'The Bourne Ultimatum',
 'Traffic',
 'These Encounters of Theirs',
 'Vincere',
 'Le qua

In [5]:
unique_director = df.drop_duplicates(subset=['movie_director'])
unique_director

,movie_name,movie_director,movie_year,movie_rank,critic_name,critic_org,critic_cn
0,Mulholland Drive,David Lynch,2001,1,Simon Abrams,Freelance film critic,US
1,In the Mood for Love,Wong Kar-wai,2000,2,Simon Abrams,Freelance film critic,US
2,The Tree of Life,Terrence Malick,2011,3,Simon Abrams,Freelance film critic,US
3,Yi Yi: A One and a Two,Edward Yang,2000,4,Simon Abrams,Freelance film critic,US
4,Goodbye to Language,Jean-Luc Godard,2014,5,Simon Abrams,Freelance film critic,US
...,...,...,...,...,...,...,...
1694,Four Lions,Christopher Morris,2010,5,Rahul Verma,Freelance film critic,UK
1699,This Is England,Shane Meadows,2006,10,Rahul Verma,Freelance film critic,UK
1724,Once,John Carney,2007,5,Alissa Wilkinson,Christianity Today,US
1761,The Hours,Stephen Daldry,2002,2,Raymond Zhou,China Daily,China


In [6]:
director_list = unique_director.movie_director.tolist()
director_list

['David Lynch',
 'Wong Kar-wai',
 'Terrence Malick',
 'Edward Yang',
 'Jean-Luc Godard',
 'Mohammad Rasoulof',
 'Raoul Ruiz',
 'Abbas Kiarostami',
 'Johnnie To',
 'Carlos Saura',
 'Michel Gondry',
 'Apichatpong Weerasethakul',
 'Hayao Miyazaki',
 'Joshua Oppenheimer',
 'Wes Anderson',
 'Jia Zhangke',
 'Gus Van Sant',
 'Kathryn Bigelow',
 'David Cronenberg',
 'Sarah Polley',
 'Martin Campbell',
 'Miguel Gomes',
 'Pablo Berger',
 'Courtney Hunt',
 'Robert Altman',
 'Christopher Nolan',
 'Guillermo Del Toro',
 'Michael Haneke',
 'Werner Herzog',
 'Cristian Mungiu',
 'Leos Carax',
 'Claude Lanzmann',
 'Paul Thomas Anderson',
 'Kenneth Lonergan',
 'Mary Harron',
 'Jessica Hausner',
 'Andrea Arnold',
 'Richard Linklater',
 'Pablo Larraín',
 'Joel and Ethan Coen',
 'Asghar Farhadi',
 'Andrew Stanton and Lee Unkrich',
 'Zhang Yimou',
 'Martin Scorsese',
 'Bong Joon-ho',
 'Paul Greengrass',
 'Steven Soderbergh',
 'Danièle Huillet and Jean-Marie Straub',
 'Marco Bellocchio',
 'Michelangelo Framm

# 2.1 Feed all the movie into IMDb

## get all the movie link

In [31]:
hrefs = []
for movie_name in moviename_list:
    one_movie = {}
    one_movie['name'] = movie_name
    name_string = movie_name.replace(" ","+").lower()
    one_movie['href'] = "https://www.imdb.com/find?q=" + name_string
    hrefs.append(one_movie)
hrefs

[{'name': 'Mulholland Drive',
  'href': 'https://www.imdb.com/find?q=mulholland+drive'},
 {'name': 'In the Mood for Love',
  'href': 'https://www.imdb.com/find?q=in+the+mood+for+love'},
 {'name': 'The Tree of Life',
  'href': 'https://www.imdb.com/find?q=the+tree+of+life'},
 {'name': 'Yi Yi: A One and a Two',
  'href': 'https://www.imdb.com/find?q=yi+yi:+a+one+and+a+two'},
 {'name': 'Goodbye to Language',
  'href': 'https://www.imdb.com/find?q=goodbye+to+language'},
 {'name': 'The White Meadows',
  'href': 'https://www.imdb.com/find?q=the+white+meadows'},
 {'name': 'Night Across the Street',
  'href': 'https://www.imdb.com/find?q=night+across+the+street'},
 {'name': 'Certified Copy',
  'href': 'https://www.imdb.com/find?q=certified+copy'},
 {'name': 'Sparrow', 'href': 'https://www.imdb.com/find?q=sparrow'},
 {'name': 'Fados', 'href': 'https://www.imdb.com/find?q=fados'},
 {'name': 'Eternal Sunshine of the Spotless Mind',
  'href': 'https://www.imdb.com/find?q=eternal+sunshine+of+the+sp

In [32]:
imdb_page = []
for href in hrefs:
    try:
        raw_html = requests.get(href['href']).content
        soup_doc = BeautifulSoup(raw_html, "html.parser")
        one_movie = {}
        text = soup_doc.find(class_='result_text').text
        link = soup_doc.find(class_='result_text').a['href']
        one_movie['name'] = href['name']
        one_movie['imdb_text'] = text
        one_movie['imdb_link'] = link
        imdb_page.append(one_movie)
    except:
        one_movie['name'] = href['name']
        one_movie['imdb_text'] = 'NaN'
        one_movie['imdb_link'] = 'NaN'
        imdb_page.append(one_movie)

In [33]:
len(imdb_page)

606

In [34]:
imdb_page

[{'name': 'Mulholland Drive',
  'imdb_text': ' Mulholland Drive (2001) ',
  'imdb_link': '/title/tt0166924/'},
 {'name': 'In the Mood for Love',
  'imdb_text': ' In the Mood for Love (2000) ',
  'imdb_link': '/title/tt0118694/'},
 {'name': 'The Tree of Life',
  'imdb_text': ' The Tree of Life (2011) ',
  'imdb_link': '/title/tt0478304/'},
 {'name': 'Yi Yi: A One and a Two',
  'imdb_text': ' Yi Yi: A One and a Two... (2000) ',
  'imdb_link': '/title/tt0244316/'},
 {'name': 'Goodbye to Language',
  'imdb_text': ' Goodbye to Language (2014) ',
  'imdb_link': '/title/tt2400275/'},
 {'name': 'The White Meadows',
  'imdb_text': ' The White Meadows (2009) ',
  'imdb_link': '/title/tt1509132/'},
 {'name': 'Night Across the Street',
  'imdb_text': ' Night Across the Street (2012) ',
  'imdb_link': '/title/tt1876360/'},
 {'name': 'Certified Copy',
  'imdb_text': ' Certified Copy (2010) ',
  'imdb_link': '/title/tt1020773/'},
 {'name': 'Sparrow',
  'imdb_text': ' Sparrow (2008) ',
  'imdb_link': 

In [ ]:
imdb_info = []
for each_movie in imdb_page:
    try:
        movie_link = "https://www.imdb.com" + each_movie['imdb_link']
        raw_html = requests.get(movie_link).content
        soup_doc = BeautifulSoup(raw_html, "html.parser")
        one_movie = {}
        one_movie['name'] = each_movie['name']
        one_movie['imdb_name'] = each_movie['imdb_text']
        one_movie['imdb_rate'] = soup_doc.find(class_ ="AggregateRatingButton__RatingScore-sc-1ll29m0-1 iTLWoV").text
        one_movie['imdb_ppl'] = soup_doc.find(class_ ="AggregateRatingButton__TotalRatingAmount-sc-1ll29m0-3 jkCVKJ").text
        one_movie['storyline'] = soup_doc.find(class_='Storyline__StorylineWrapper-sc-1b58ttw-0 iywpty').find(class_='ipc-html-content ipc-html-content--base').text
        imdb_info.append(one_movie)              
    except:
        one_movie['imdb_name'] = each_movie['name']
        one_movie['imdb_name'] = each_movie['imdb_text']
        one_movie['imdb_rate']  = 'NaN'
        one_movie['imdb_ppl']  = 'NaN'
        one_movie['storyline']  = 'NaN'
        imdb_page.append(one_movie)

In [39]:
pd.DataFrame(imdb_info).to_csv('1215_movie_test1.csv',index=False)

In [38]:
len(imdb_info)

583

## 2.2 Feed all the directors into IMDb

In [7]:
dhrefs = []
for director_name in director_list:
    director = {}
    director['director_name'] = director_name
    dname_string = director_name.replace(" ","+").lower()
    dhref = "https://www.imdb.com/find?q=" + dname_string
    director['link'] = dhref
    dhrefs.append(director)
dhrefs

[{'director_name': 'David Lynch',
  'link': 'https://www.imdb.com/find?q=david+lynch'},
 {'director_name': 'Wong Kar-wai',
  'link': 'https://www.imdb.com/find?q=wong+kar-wai'},
 {'director_name': 'Terrence Malick',
  'link': 'https://www.imdb.com/find?q=terrence+malick'},
 {'director_name': 'Edward Yang',
  'link': 'https://www.imdb.com/find?q=edward+yang'},
 {'director_name': 'Jean-Luc Godard',
  'link': 'https://www.imdb.com/find?q=jean-luc+godard'},
 {'director_name': 'Mohammad Rasoulof',
  'link': 'https://www.imdb.com/find?q=mohammad+rasoulof'},
 {'director_name': 'Raoul Ruiz',
  'link': 'https://www.imdb.com/find?q=raoul+ruiz'},
 {'director_name': 'Abbas Kiarostami',
  'link': 'https://www.imdb.com/find?q=abbas+kiarostami'},
 {'director_name': 'Johnnie To',
  'link': 'https://www.imdb.com/find?q=johnnie+to'},
 {'director_name': 'Carlos Saura',
  'link': 'https://www.imdb.com/find?q=carlos+saura'},
 {'director_name': 'Michel Gondry',
  'link': 'https://www.imdb.com/find?q=michel+

In [8]:
len(dhrefs)

412

In [9]:
director_page = []
for dhref in dhrefs:
    try:
        raw_html = requests.get(dhref['link']).content
        soup_doc = BeautifulSoup(raw_html, "html.parser")
        one_director = {}
        one_director['name'] = dhref['director_name']
        link = soup_doc.find(class_='result_text').a['href']
        one_director['imdb_link'] = link
        director_page.append(one_director)
    except:
        one_director['name'] = dhref['director_name']
        one_director['imdb_link'] = 'NaN'
        director_page.append(one_director)

In [15]:
len(director_page)

412

In [16]:
d_info = []
for each_d in director_page:
    try:
        d_link = "https://www.imdb.com" + each_d['imdb_link']
        raw_html = requests.get(d_link).content
        soup_doc = BeautifulSoup(raw_html, "html.parser")
        one_d = {}
        one_d['name'] = each_d['name']
        one_d['birth_place'] = soup_doc.find(id='name-born-info').find_all('a')[-1].text
        d_info.append(one_d)
    except:
        one_d['name'] = each_d['name']
        one_d['birth_place'] = 'NaN'
        d_info.append(one_d)

In [17]:
d_info

[{'name': 'David Lynch', 'birth_place': 'Missoula, Montana, USA'},
 {'name': 'Wong Kar-wai', 'birth_place': 'Shanghai, China'},
 {'name': 'Terrence Malick', 'birth_place': 'Ottawa, Illinois, USA'},
 {'name': 'Edward Yang', 'birth_place': 'Shanghai, China'},
 {'name': 'Jean-Luc Godard', 'birth_place': 'Paris, France'},
 {'name': 'Mohammad Rasoulof', 'birth_place': 'Shiraz, Iran'},
 {'name': 'Raoul Ruiz', 'birth_place': 'Puerto Montt, Chile'},
 {'name': 'Abbas Kiarostami', 'birth_place': 'Tehran, Iran'},
 {'name': 'Johnnie To', 'birth_place': 'Hong Kong'},
 {'name': 'Carlos Saura', 'birth_place': 'Huesca, Aragón, Spain'},
 {'name': 'Michel Gondry',
  'birth_place': 'Versailles, Seine-et-Oise [now Yvelines], France'},
 {'name': 'Apichatpong Weerasethakul', 'birth_place': 'Bangkok, Thailand'},
 {'name': 'Hayao Miyazaki', 'birth_place': 'Tokyo, Japan'},
 {'name': 'Joshua Oppenheimer', 'birth_place': 'Texas, USA'},
 {'name': 'Wes Anderson', 'birth_place': 'Houston, Texas, USA'},
 {'name': 'J

In [19]:
pd.DataFrame(d_info).to_csv('director_info.csv',index=False)